In [22]:
paragraph="""Susan B. Anthony, Woman’s Rights to the Suffrage (1873)
At a time when women were denied basic rights, such as the right to vote or own property, Susan B. Anthony emerged as a fierce champion for women’s suffrage. She advocated relentlessly throughout the latter part of the 19th century.

Following an arrest in her hometown of Rochester, New York, for illegally casting a ballot during the 1872 presidential election, she toured the nation. At venues ranging from crowded town squares to courtrooms, she boldly articulated the injustices faced by women under existing laws. Anthony proclaimed that they too deserved full citizenship and representation within a democratic society. Although she didn’t live to witness the passage of the 19th Amendment—finally granting women nationwide voting rights in 1920—her impassioned pleas inspired countless others (women AND men) to join the cause. This furthered the advancement toward gender equality for generations to come.

Franklin Delano Roosevelt, “The only thing we have to fear is fear itself” (1933):
On March 4th, 1933, newly-elected President Franklin D. Roosevelt assumed office. He faced the daunting task of restoring hope to a nation gripped by the devastating Great Depression. By that year, the Depression had paralyzed both national and global economies. Delivered with characteristic calm assurance during his inaugural address, Roosevelt famously proclaimed “the only thing we have to fear is fear itself”. He then called upon his countrymen to rally together as they faced a multitude of challenges.

The strength and resoluteness embodied within those twelve famous words instilled confidence during a time characterized by unprecedented economic despair and widespread social anxiety. His speech ignited a collective sense of resilience within American citizens. This proved vital in mobilizing support for his ambitious “New Deal” program. This program contained a series of government reforms aimed at alleviating the devastating effects of the economic downturn. Today, it stands as one of the most enduring examples of inspirational rhetoric in US history. This demonstrates how impactful a leader’s words can be when confronting periods of uncertainty.

John F. Kennedy’s First Inaugural Address (1961):
Considered a high-water mark in political speechmaking, President John F. Kennedy’s 1961 inaugural address perfectly captured the spirit of a generation on the verge of profound cultural shifts.

His words offered both soaring rhetoric about American ideals while simultaneously presenting a pragmatic vision for engaging global challenges. These challenges ranged from reducing poverty to mitigating the risk of nuclear war during an age fraught with Cold War tensions. Delivered just two years before his assassination in 1963, it remains one of the best inaugural addresses ever given. This is due to the late president’s mastery of rhetoric and delivery and for its message of hope and international cooperation which continues to resonate more than sixty years after its initial delivery.

Ronald Reagan’s Address at the Berlin Wall (1987):
In June of 1987, at the height of Cold War tensions, US President Ronald Reagan addressed Soviet premier Mikhail Gorbachev from Brandenburg Gate, near one of the most visible symbols of the Iron Curtain. The Berlin Wall had, for nearly three decades, served as a grim physical reminder of Europe’s division.

Reagan boldly challenged Gorbachev—and those watching around the world—“Mr. Gorbachev, tear down this wall.”. This simple yet profound demand for an end to Soviet domination reverberated globally. It galvanized pro-democracy movements and symbolized the growing desire for reunification between East and West Germany. Just two years later, on November 9th, 1989, crowds began gathering at sections of the wall. East and West Berliners alike celebrated the reunification of their city. The event still serves as a poignant example of a speech’s influence extending far beyond its immediate context. It powerfully impacted both geopolitical realities and individual lives around the world.

Find Out Exactly How Much You Could Make As a Paid Speaker

Use The Official Speaker Fee Calculator to tell you what you should charge for your first (or next) speaking gig — virtual or in-person! 
"""

In [23]:
from gensim.models import Word2Vec,KeyedVectors

In [24]:
import gensim.downloader as api

In [25]:
w=api.load('word2vec-google-news-300')

In [26]:
import nltk
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


In [27]:
original_sentences=nltk.sent_tokenize(paragraph)

In [28]:
lemmatizer=WordNetLemmatizer()

In [29]:
stop_words=stopwords.words('english')

In [30]:
corpus = []
for sentence in original_sentences:
    review = re.sub('[^a-zA-Z]', ' ', sentence)  # Fixed regex
    review = review.lower()
    words = review.split()
    processed = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]  # Fixed word iteration
    review = ' '.join(processed)
    corpus.append(review)

In [31]:
words=[]
for i in range(len(corpus)):
    wor=nltk.word_tokenize(corpus[i])
    words.append(wor)

In [32]:
sentence_vectors = []
for lst in words:
    word_vecs = []
    for word in lst:
        if word in w:
            word_vecs.append(w[word])
    if word_vecs:
        sentence_vec = np.mean(word_vecs, axis=0)
        sentence_vectors.append(sentence_vec)



In [33]:
import numpy as np

In [34]:
doc_vector = np.mean(sentence_vectors, axis=0)  # (300,)



In [35]:
doc_vector.reshape(-1,1)

array([[ 0.0334946 ],
       [ 0.04860333],
       [ 0.02711649],
       [ 0.08644563],
       [-0.04148689],
       [-0.06307489],
       [ 0.03123738],
       [-0.09762678],
       [ 0.11923043],
       [ 0.07833925],
       [-0.04684241],
       [-0.09928079],
       [-0.01189533],
       [ 0.06221694],
       [-0.02745553],
       [ 0.07468227],
       [ 0.02607269],
       [ 0.10546774],
       [ 0.01313547],
       [-0.08035589],
       [-0.00798677],
       [ 0.04721424],
       [ 0.01551921],
       [-0.02522974],
       [ 0.01165813],
       [-0.02395393],
       [-0.06152791],
       [ 0.02923621],
       [ 0.04924667],
       [-0.0138604 ],
       [-0.01757367],
       [-0.05447789],
       [-0.03341477],
       [ 0.02326587],
       [ 0.02090634],
       [ 0.01499594],
       [ 0.02254788],
       [-0.00552692],
       [ 0.05084794],
       [ 0.04045631],
       [ 0.05018618],
       [-0.02244015],
       [ 0.09907126],
       [ 0.018623  ],
       [-0.03195556],
       [-0

In [36]:
from sklearn.metrics.pairwise import cosine_similarity

In [38]:
sentence_vectors = np.array(sentence_vectors)

# Step 3: Compute document vector (average of sentence vectors)
doc_vector = np.mean(sentence_vectors, axis=0).reshape(1, -1)

# Step 4: Compute cosine similarity of each sentence vector with document vector
similarities = cosine_similarity(sentence_vectors, doc_vector).flatten()

# Step 5: Select top N sentences for summary
N = 30
top_indices = similarities.argsort()[-N:][::-1]  # indices of top N sentences by similarity
top_indices = sorted(top_indices)  # sort to preserve original order

# Step 6: Create summary by joining original sentences
summary = " ".join([original_sentences[i] for i in top_indices])

print(summary)

Susan B. Anthony, Woman’s Rights to the Suffrage (1873)
At a time when women were denied basic rights, such as the right to vote or own property, Susan B. Anthony emerged as a fierce champion for women’s suffrage. She advocated relentlessly throughout the latter part of the 19th century. Following an arrest in her hometown of Rochester, New York, for illegally casting a ballot during the 1872 presidential election, she toured the nation. At venues ranging from crowded town squares to courtrooms, she boldly articulated the injustices faced by women under existing laws. Anthony proclaimed that they too deserved full citizenship and representation within a democratic society. Although she didn’t live to witness the passage of the 19th Amendment—finally granting women nationwide voting rights in 1920—her impassioned pleas inspired countless others (women AND men) to join the cause. This furthered the advancement toward gender equality for generations to come. Franklin Delano Roosevelt, “Th